# 키워드와 문장만으로 이루어진 판례를 속성 추가해서 input 데이터 만들기

In [1]:
import json
from tqdm import tnrange, tqdm_notebook

In [2]:
f = open('C:/Users/jayts/OneDrive/바탕 화면/Tictag/json/KAS/removed_duplicate_panrye_통합본_2021_8_10.json', 'r', encoding='UTF8')    # type3를 통해 추출된 json파일 읽어들이기
# f = open('C:/Users/jayts/OneDrive/바탕 화면/Tictag/json/KAS/removed_dup_example.json', 'r', encoding='UTF8')    # type3를 통해 추출된 json파일 읽어들이기
sent_example = json.load(f)

In [3]:
# 단어 하나가 문장에 여러번 등장했을때, 모든 위치를 리스트로 뽑아내는 함수.
# list씌워야함

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches
        
# ex) list(find_all('안녕안녕하세요', '안녕')) --> [0,2]

In [4]:
# 숫자 리스트 받아와서 이에 맞게 다른 리스트의 순서 바꿔주는 함수.
def sort_word_wordOcc(word, wordOccurence):    # 순서 바꿀 단어 리스트, 순서 알려주는 리스트
#     try:
    if len(word) == len(wordOccurence):
        result = {}
        for i in range(len(word)):
            result.update({wordOccurence[i]['start']:word[i]})    # result = {wordOccurence:['start']:호환}

        sorted_result = sorted(result.items())     # ex) [('1','원심'), (17:'원심'), (127:'호환'), (172:'호환'), (320:'원심')]
        result_word = []
        result_wordOcc = []

        for i in range(0, len(word)):
            result_word.append(sorted_result[i][1])
            result_wordOcc.append({'start':sorted_result[i][0], 'end':sorted_result[i][0]+len(sorted_result[i][1])+1})
        
    else:
        raise Exception('word와 word_Occurence 간 크기가 다릅니다.')
        
#     except Exception as e:
#         print("예외가 발생했습니다.", e)
        
    return result_word, result_wordOcc

In [5]:
# 중복제거
def remove_overlap(overlapped_list):
    result = []
    for i in overlapped_list:
        if i not in result:
            result.append(i)
            
    return result

In [6]:
# 문장에 ^^ 태그하고 sentence와 word 업데이트.
type2_A_input = {}
tag = '^'
key_list = sorted(list(sent_example.keys()), key=len)    # 어려운 단어 리스트. 글자수가 더 많은 키워드를 먼저 ^^로 태깅하면 문제가 생김. 그래서 오름차순 정렬함
keyword_pos = []
for i in tnrange(len(key_list)):
    type2_A_input.update({key_list[i]:[]})
    
    for j in range(len(sent_example[key_list[i]])):
        sentence = sent_example[key_list[i]][j]
        type2_A_input[key_list[i]].append({'sentence':None, 'word':[], 'wordOccurence':[]})
        
        # 키데이터 중 문장에 있는 키워드 찾아내기
        for k in range(len(key_list)):    # 모든 키워드에 대해 시행
            tagged_keyword = tag + key_list[k] + tag
            if sentence[0:len(key_list[k])] == key_list[k]:    # 문장 첫 단어로 키워드가 있을 때..
                sentence = sentence.replace(key_list[k], tagged_keyword)
                count = len(list(find_all(sentence, key_list[k])))
                type2_A_input[key_list[i]][j].update({'sentence':sentence})
                for l in range(count):
                    type2_A_input[key_list[i]][j]['word'].append(key_list[k])
                
                
            elif sentence.find(' '+key_list[k]) != -1:    # 문장 첫 단어로 키워드가 있을 때..
                sentence = sentence.replace(key_list[k], tagged_keyword)
                count = len(list(find_all(sentence, key_list[k])))
                type2_A_input[key_list[i]][j].update({'sentence':sentence})
                for l in range(count):
                    type2_A_input[key_list[i]][j]['word'].append(key_list[k])
                
            else:
                pass            

<ipython-input-6-849b702ad83b>:6: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(key_list)):


  0%|          | 0/2732 [00:00<?, ?it/s]

In [7]:
# 태그된 문장을 참고하여 wordOccurence 업데이트.
for i in tnrange(len(key_list)):
    for j in range(len(sent_example[key_list[i]])):
        key_list_small = type2_A_input[key_list[i]][j]['word']
        key_list_small_not_overlapped = remove_overlap(key_list_small)
        for k in range(len(key_list_small_not_overlapped)):    # 문장 속 존재하는 키워드들에 대해 시행
            sentence = type2_A_input[key_list[i]][j]['sentence']
            start = list(find_all(sentence, key_list_small_not_overlapped[k]))
            for l in range(len(start)):
                type2_A_input[key_list[i]][j]['wordOccurence'].append({'start':start[l]-1, 'end':start[l]+len(key_list[k])+1})

<ipython-input-7-a2851908ec07>:2: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(key_list)):


  0%|          | 0/2732 [00:00<?, ?it/s]

In [8]:
# word와 wordOccurence 직관적으로 정렬.
for i in tnrange(len(key_list)):
    for j in range(len(sent_example[key_list[i]])):
        keyword = type2_A_input[key_list[i]][j]['word']
        tagged_keyword_pos = type2_A_input[key_list[i]][j]['wordOccurence']
        result_word, result_wordOcc = sort_word_wordOcc(keyword, tagged_keyword_pos)
        type2_A_input[key_list[i]][j]['word'] = result_word
        type2_A_input[key_list[i]][j]['wordOccurence'] = result_wordOcc

<ipython-input-8-891784168793>:2: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(key_list)):


  0%|          | 0/2732 [00:00<?, ?it/s]

In [9]:
# json 파일로 저장
with open('C:/Users/jayts/OneDrive/바탕 화면/Tictag/json/KAS/type2_A_input.json', 'w', encoding='UTF8') as json_file:
# with open('C:/Users/jayts/OneDrive/바탕 화면/Tictag/json/KAS/type2_A_input_example.json', 'w', encoding='UTF8') as json_file:
    json.dump(type2_A_input, json_file, indent=4, ensure_ascii=False,)